# 파일 실행 : 다른 코드라인은 실행하지 않고 아래의 코드라인만 실행
- !python custom_yolov5_detect.py --weights C:\Users\kimdu\Downloads\best.pt --conf 0.5 --source 1

## best.pt를 다운로드 후 다운로드 한 위치로 C:\Users\kimdu\Downloads\best.pt를 수정 후 실행하시면 됩니다.


In [4]:
# clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5  # clone repo

fatal: Too many arguments.

usage: git clone [<options>] [--] <repo> [<dir>]

    -v, --verbose         be more verbose
    -q, --quiet           be more quiet
    --progress            force progress reporting
    --reject-shallow      don't clone shallow repository
    -n, --no-checkout     don't create a checkout
    --bare                create a bare repository
    --mirror              create a mirror repository (implies bare)
    -l, --local           to clone from a local repository
    --no-hardlinks        don't use local hardlinks, always copy
    -s, --shared          setup as shared repository
    --recurse-submodules[=<pathspec>]
                          initialize submodules in the clone
    --recursive ...       alias of --recurse-submodules
    -j, --jobs <n>        number of submodules cloned in parallel
    --template <template-directory>
                          directory from which templates will be used
    --reference <repo>    reference repository
    --refere

### 파일을 저장할 위치로 이동한다.

In [3]:
%cd /Users/kimdu/yolov5

C:\Users\kimdu\yolov5


In [4]:
!git reset --hard 064365d8683fd002e9ad789c1e91fa3d021b44f0

HEAD is now at 064365d Update parse_opt() in export.py to work as in train.py (#10789)


In [4]:
# !pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch

from IPython.display import Image, clear_output  # to display images
from utils.downloads import attempt_download  # to download models/datasets

In [5]:
import torch

### Using torch 1.7.1+cu110 가 나와야 GPU를 사용하는 중이다.
- GPU를 사용하지 않는다면 학습에 어마무시한 시간이 걸리므로 반드시 이 부분 확인할 것

In [22]:
# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 1.7.1+cu110 _CudaDeviceProperties(name='NVIDIA GeForce RTX 3070 Laptop GPU', major=8, minor=6, total_memory=8191MB, multi_processor_count=40)


In [7]:
%cd /Users/kimdu/yolov5

C:\Users\kimdu\yolov5


### Roboflow를 활용하여 라벨링을 진행하였으므로 roboflow를 install 한다.

In [8]:
!pip install roboflow

  Obtaining dependency information for roboflow from https://files.pythonhosted.org/packages/77/1c/b6a244353965f74b3dad9c2a8726856d3e0eeb3e2e5c9e65e56dc6e7188b/roboflow-1.1.4-py3-none-any.whl.metadata
  Using cached certifi-2022.12.7-py3-none-any.whl (155 kB)
  Using cached chardet-4.0.0-py2.py3-none-any.whl (178 kB)
  Using cached cycler-0.10.0-py2.py3-none-any.whl (6.5 kB)
  Using cached idna-2.10-py2.py3-none-any.whl (58 kB)
  Obtaining dependency information for kiwisolver>=1.3.1 from https://files.pythonhosted.org/packages/ca/c1/1f986c8119c0c57c2bd71d1941da23332c38ee2c90117e46dff4358b70f7/kiwisolver-1.4.5-cp39-cp39-win_amd64.whl.metadata
  Using cached kiwisolver-1.4.5-cp39-cp39-win_amd64.whl.metadata (6.5 kB)
  Obtaining dependency information for matplotlib from https://files.pythonhosted.org/packages/c9/46/6cbaf20f5bd0e7c1d204b45b853c2cd317b303fada90245f2825ecca47de/matplotlib-3.7.2-cp39-cp39-win_amd64.whl.metadata
  Using cached matplotlib-3.7.2-cp39-cp39-win_amd64.whl.metadat

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboard 2.6.0 requires google-auth<2,>=1.6.3, but you have google-auth 2.6.0 which is incompatible.
tensorflow-gpu 2.9.0 requires tensorboard<2.10,>=2.9, but you have tensorboard 2.6.0 which is incompatible.
tensorflow-gpu 2.9.0 requires tensorflow-estimator<2.10.0,>=2.9.0rc0, but you have tensorflow-estimator 2.6.0 which is incompatible.


### 아래의 api_key와 project는 직접 설정한 값이다.
- roboflow에서 확인할 수 있다.
- roboflow에서 container_turn_buckle_degree를 검색하면 사용할 수 있다.
- api_key의 경우 개인의 api_key를 사용

In [9]:
from roboflow import Roboflow
rf = Roboflow(api_key="")
project = rf.workspace("citylights").project("container_turn_buckle_degree")
dataset = project.version(1).download("yolov5")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Container_turn_buckle_degree-1 in yolov5pytorch:: 100%|█| 956/956 [00:00<00:00, 1008.


In [10]:
%cd /Users/kimdu/yolov5

%pycat {dataset.location}/data.yaml

C:\Users\kimdu\yolov5


names:
- 0deg
- 120deg
- 150deg
- 180deg
- 30deg
- 60deg
- 90deg
- Loose
- Tight
nc: 9
roboflow:
  license: CC BY 4.0
  project: container_turn_buckle_degree
  url: https://universe.roboflow.com/citylights/container_turn_buckle_degree/dataset/1
  version: 1
  workspace: citylights
test: ../test/images
train: Container_turn_buckle_degree-1/train/images
val: Container_turn_buckle_degree-1/valid/images


In [11]:
# define number of classes based on YAML
import yaml
with open(dataset.location + "/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

### /Users/kimdu/yolov5/models/custom_yolov5s.yaml
- 위 부분을 본인이 yaml 파일을 저장한 위치로 바꿔주면 된다.

In [12]:
#this is the model configuration we will use for our tutorial
%pycat /Users/kimdu/yolov5/models/yolov5s.yaml

# YOLOv5 🚀 by Ultralytics, GPL-3.0 license

# Parameters
nc: 80  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]]

In [13]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

### /Users/kimdu/yolov5/models/custom_yolov5s.yaml
- 위 부분을 본인이 yaml 파일을 저장한 위치로 바꿔주면 된다.

In [14]:
%%writetemplate /Users/kimdu/yolov5/models/custom_yolov5s.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

### /Users/kimdu/yolov5/models/custom_yolov5s.yaml
- 위 부분을 본인이 yaml 파일을 저장한 위치로 바꿔주면 된다.

In [ ]:
%%time
%cd /Users/kimdu/yolov5/
!python train.py --img 640 --batch 32 --epochs 250 --data {dataset.location}/data.yaml --cfg /Users/kimdu/yolov5/models/custom_yolov5s.yaml --weights '' --name yolov5s_results  --cache

### source를 custom하여 웹캠에서 구동할 것인지 혹은 노트북의 웹캠으로 구동할 것인지 결정한다
- source 1 : 연결된 웹캠을 사용
- source 0 : 노트북의 웹캠을 사용

In [2]:
!python custom_yolov5_detect.py --weights C:\Users\kimdu\Downloads\best.pt --conf 0.5 --source 1

Final Rotate Value: 0
Final Rotate Value: 0
Final Rotate Value: 0
Final Rotate Value: 0
Final Rotate Value: 0
Final Rotate Value: 0
Final Rotate Value: 0
The most detected class in the last 5 seconds is: 30deg
Final Rotate Value: 0


detect3: weights=['C:\\Users\\kimdu\\Downloads\\best.pt'], source=1, data=data\coco128.yaml, imgsz=[640, 640], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5  v7.0-72-g064365d Python-3.9.17 torch-1.7.1+cu110 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 8192MiB)

Fusing layers... 
custom_YOLOv5s summary: 182 layers, 7268094 parameters, 0 gradients
1/1: 1...  Success (inf frames 640x480 at 30.00 FPS)

0: 480x640 1 0deg, 861.2ms
0: 480x640 1 0deg, 24.5ms
0: 480x640 1 0deg, 25.2ms
0: 480x640 1 0deg, 21.7ms
0: 480x640 1 0deg, 20.6ms
0: 480x640 1 0deg, 17.6ms
0: 480x640 1 0deg, 18.1ms
0: 480x640 1 0deg, 24.0ms
0: 480x640 1 0deg, 21.2ms
0: 480x640 1 0deg, 23.0ms
0: 480x640 1 0deg, 24.2ms